In [ ]:
%pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import spacy
import random

from spacy.util import minibatch
from spacy.pipeline.textcat_multilabel import DEFAULT_MULTI_TEXTCAT_MODEL

from sklearn.model_selection import train_test_split

ModuleNotFoundError: ignored

In [ ]:
import pandas as pd
import spacy
from sklearn.model_selection import train_test_split

In [ ]:
posts_df = pd.read_csv('./posts_scores_dates.csv')
train_df, test_df = train_test_split(posts_df, random_state=20, stratify=posts_df['viral'])

In [ ]:
df= train_df
nlp = spacy.blank("en")
config = {
   "threshold": 0.5,
   "model": DEFAULT_MULTI_TEXTCAT_MODEL,
}

textcat = nlp.add_pipe("textcat", config=config)
textcat.add_label("viral")
textcat.add_label("not_viral")
train_texts = df['title'].values
train_labels = [{'cats': {'viral': label == 1,
                          'not_viral': label == 0}}
                for label in df['viral']]
train_data = list(zip(train_texts, train_labels))
spacy.util.fix_random_seed(1)
optimizer = nlp.initialize()
# Create the batch generator with batch size = 8
batches = minibatch(train_data, size=8)

# Iterate through minibatches
TRAIN_DATA = train_data
random.shuffle(TRAIN_DATA)
losses = {}
for batch in minibatch(TRAIN_DATA, size=8):
    for text, annotations in batch:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], drop=0.35, sgd=optimizer, losses=losses)

import random

random.seed(1)
spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

losses = {}
for epoch in range(5):
    random.shuffle(train_data)
    # Create the batch generator with batch size = 8
    batches = minibatch(train_data, size=8)
    # Iterate through minibatches
    for batch in batches:
        for text, annotations in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], drop=0.35, sgd=optimizer, losses=losses)
    print(losses)


{'textcat': 165.59590898674847}
{'textcat': 327.45672613504274}
{'textcat': 488.6894359766405}
{'textcat': 645.8258839023949}
{'textcat': 792.9818615397359}


In [ ]:
texts = test_df['title']
docs = [nlp.tokenizer(text) for text in texts]

In [ ]:
# Use textcat to get the scores for each doc
textcat = nlp.get_pipe('textcat')
scores = textcat.predict(docs)

print(scores)
predicted_labels = scores.argmax(axis=1)
print([textcat.labels[label] for label in predicted_labels])

[[0.00227116 0.9975569 ]
 [0.00259717 0.99686533]
 [0.0053241  0.9950783 ]
 ...
 [0.00325172 0.99643075]
 [0.00237572 0.9974764 ]
 [0.00236876 0.9974564 ]]
['not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral', 'not_viral'

In [ ]:
len(textcat.predict(docs))

100

In [ ]:
[textcat.labels[label] for label in predicted_labels][:2]

['not_viral', 'not_viral']

In [ ]:
textcat.predict([nlp.tokenizer(text) for text in test_df['title']]).argmax(axis=1)

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
[textcat.labels[label] for label in textcat.predict([nlp.tokenizer(text) for text in test_df['title']]).argmax(axis=1)][:2]

['not_viral', 'not_viral']

In [ ]:
textcat.score([nlp.tokenizer(text) for text in test_df['title']])

AttributeError: ignored

Code adapted from https://github.com/roisinod/SpaCy-intro/blob/260ac6366c2d9b7b80ca65f579f34683361a1ea5/SpaCy%20textcat.py

Below is code that was used to try to generate predictions. Errors were found when trying to run the second command line prompt with a message indicating categories were not properly instantiated.

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
### turn data into tuples with text and label
train_data = tuple(zip(train_df.title.tolist(), train_df.viral.to_list()))
test_data = tuple(zip(test_df.title.tolist(), test_df.viral.to_list()))

In [ ]:
### function to get docs
def document(data):
#Creating empty list called "text"
  text = []
  for doc, label in nlp.pipe(data, as_tuples = True):
    if (label== 1):
      doc.cats['viral'] = 1
    elif (label==0):
      doc.cats['not_viral'] = 1
#Adding the doc into the list 'text'
      text.append(doc)
  return(text)

In [ ]:
#passing the train dataset into function 'document'
train_docs = document(train_data)

#Creating binary document using DocBin function in spaCy
doc_bin = DocBin(docs = train_docs)

#Saving the binary document as train.spacy
doc_bin.to_disk("train.spacy")

NameError: ignored

In [ ]:
#passing the test dataset into function 'document'
test_docs = document(test)
doc_bin = DocBin(docs = test_docs)
doc_bin.to_disk("valid.spacy")

In [ ]:
doc_bin_train = DocBin(docs=train_docs)
doc_bin_test = DocBin(docs=test_docs)

!python -m spacy init fill-config base_config.cfg config.cfg

!python -m spacy train config.cfg --output ./output --paths.train doc_bin_train --paths.dev doc_bin_test